# Faucet & Wallet analysis
Let's take an initial look at the faucets and people that donate to them.

In [1]:
# some standard imports...
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import web3
import os
import sys
import json
import time
import requests

# load ../secrets.json
with open("../secrets.json") as f:
    secrets = json.load(f)
    f.close()

faucets = {
    "Goerli": {
        "goerliPOW": "0xA5058fbcD09425e922E3E9e78D569aB84EdB88Eb",
    },
}

alchemy_url = secrets["alchemy_https"]
etherscan_key = secrets["etherscan_key"]
web3 = web3.Web3(web3.HTTPProvider(alchemy_url))

In [2]:
import json
import time

import requests

# Place your own Etherscan API key here
ETHERSCAN_API_KEY = etherscan_key
ETHERSCAN_API_KEY = "JKXR76EA1NK9XSKES1T2BNYBZ41T3BJ2JH"
ADDRESS = "0xA5058fbcD09425e922E3E9e78D569aB84EdB88Eb"
URL = "https://api-goerli.etherscan.io/api"  # Goerli testnet


def get_all_transactions(address):
    """
    Get all transactions for an address

    """
    params = {
        "module": "account",
        "action": "txlist",
        "address": address,
        "startblock": 0,
        "endblock": 999999999,
        "sort": "asc",
        "apikey": ETHERSCAN_API_KEY,
    }
    headers = {"User-Agent": "Chrome/41.0.2228.0"}

    # print request url
    print(URL + "?" + "&".join("{}={}".format(k, v) for k, v in params.items()))
    response = requests.get(URL, params=params, headers=headers)
    if response.status_code == 200:
        data = response.json()
        if data["status"] == "0":
            print(f"Error: {data}")
        else:
            return data["result"]
    else:
        print("Error:", response.status_code)
        print(response.text)
        return None


txs = get_all_transactions("0xA5058fbcD09425e922E3E9e78D569aB84EdB88Eb")
print(f"{len(txs)} transactions found")

https://api-goerli.etherscan.io/api?module=account&action=txlist&address=0xA5058fbcD09425e922E3E9e78D569aB84EdB88Eb&startblock=0&endblock=999999999&sort=asc&apikey=JKXR76EA1NK9XSKES1T2BNYBZ41T3BJ2JH
176 transactions found


In [3]:
# let's take a look at what one of these transactions looks like
tx = txs[-1]
print(tx)

{'blockNumber': '7275387', 'timeStamp': '1658559737', 'hash': '0x673838e1fb3e7affcdc07d3971b3083ce3b4dadbb281aacf775c00aa3fc0469a', 'nonce': '40742', 'blockHash': '0xe8ea77899773b78a4103292955be06c11fed331607faa21e8634900c2a3bb3bd', 'transactionIndex': '5', 'from': '0x6cc9397c3b38739dacbfaa68ead5f5d77ba5f455', 'to': '0xa5058fbcd09425e922e3e9e78d569ab84edb88eb', 'value': '0', 'gas': '300000', 'gasPrice': '1500000007', 'isError': '0', 'txreceipt_status': '1', 'input': '0x2e1a7d4d00000000000000000000000000000000000000000000000ad78ebc5ac6200000', 'contractAddress': '', 'cumulativeGasUsed': '423231', 'gasUsed': '89319', 'confirmations': '1099'}


In [6]:
txs_df = pd.DataFrame(txs)
# convert value to big float
txs_df["value"] = txs_df["value"].astype(np.float128)
txs_df.head()

,blockNumber,timeStamp,hash,nonce,blockHash,transactionIndex,from,to,value,gas,gasPrice,isError,txreceipt_status,input,contractAddress,cumulativeGasUsed,gasUsed,confirmations
0,7188368,1657245172,0x5edc9536ad1d7b332293e833c789a227d6c2dbbd793d...,40,0x7c42335cfede528164bd9fb1ecd4eef3273262752ae0...,2,0x332e43696a505ef45b9319973785f837ce5267b9,0xa5058fbcd09425e922e3e9e78d569ab84edb88eb,0.000000e+00,68941,2500000008,0,1,0x04acbb990000000000000000000000006cc9397c3b38...,,210442,68941,88118
1,7188449,1657246388,0x596a1995020ac096868ebb250095f861db5475b70caf...,41,0x34f65b2466d6610995277f585ea3d9a179e68ef91ae5...,9,0x332e43696a505ef45b9319973785f837ce5267b9,0xa5058fbcd09425e922e3e9e78d569ab84edb88eb,2.000000e+19,31582,1825346007,0,1,0x,,5245783,21055,88037
2,7188454,1657246463,0xe0708423919d14e3eba95ea6774c8e2ce605d4986ab7...,42,0xccdf630a68c5f5946dddae0cccc853444abbe1ebb485...,4,0x332e43696a505ef45b9319973785f837ce5267b9,0xa5058fbcd09425e922e3e9e78d569ab84edb88eb,0.000000e+00,36058,2500000007,0,1,0xc1756a2c0000000000000000000000001ea692e68a77...,,392915,33849,88032
3,7188461,1657246568,0x20231609d50318d8586aa0855d0fbe7409d1a613e464...,31932,0x12d2043fe19010d8aab800d9e893c43cf74f6dbc1a0d...,10,0x6cc9397c3b38739dacbfaa68ead5f5d77ba5f455,0xa5058fbcd09425e922e3e9e78d569ab84edb88eb,4.000000e+21,21000,1500000007,1,0,0x,,1292454,21000,88025
4,7188470,1657246706,0x0e6ddf809213db13569e2b817c61c6996dab99c591a4...,10,0x9cfabcb0b0d65e6cc98a21051419c45d9856967d2b03...,2,0x7f31d555b06d74df72becaa7b34ef88248894dae,0xa5058fbcd09425e922e3e9e78d569ab84edb88eb,5.000000e+21,31582,1940000007,0,1,0x,,101672,21055,88016


In [12]:
# let's rank the people that have contributed to this faucet
addresses = {}
for idx, tx in txs_df.iterrows():
    a_from = tx["from"]
    if a_from not in addresses:
        addresses[a_from] = 0

    value = tx["value"] / 10**18
    addresses[a_from] += value
# turn into df and order by value
addresses_df = pd.DataFrame.from_dict(addresses, orient="index")
addresses_df.columns = ["value"]
addresses_df = addresses_df.sort_values(by="value", ascending=False)
addresses_df.head()

,value
0x7f31d555b06d74df72becaa7b34ef88248894dae,30999.991167
0x6cc9397c3b38739dacbfaa68ead5f5d77ba5f455,4000.212000
0x332e43696a505ef45b9319973785f837ce5267b9,29.408833
0x38ffe5587036721b264057a45be457d1ce2723b9,8.750000
0x0bba9956e4daf62b142228c631af56a59946291a,1.000000
